In [1]:
!pip install -qU langchain huggingface_hub Chromadb pyPDF sentence-transformers accelerate ctransformers datasets openai tiktoken lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.

In [4]:
!mkdir -p blog_posts

#download the documents from https://www.dropbox.com/scl/fi/ulbt145sthizf2nazey49/langchain_blog_posts.zip?rlkey=9unhw0vukhlwacahmpnk5m591&dl=0
#and upload the files into blog_posts folder

!unzip -q /content/blog_posts/langchain_blog_posts.zip -d blog_posts

replace blog_posts/blog.langchain.dev_titantakeoff-x-langchain-supercharged-local-inference-for-llms-2_.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
replace blog_posts/blog.langchain.dev_gpt-researcher-x-langchain_.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: All


In [5]:
import os

os.environ["OPENAI_API_KEY"] = "**********"

In [6]:
!pip show langchain

Name: langchain
Version: 0.0.352
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [7]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever

## Text Splitting & Docloader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader


BGE Embeddings - BGE is short for BAAI(Beijing Academy of Artificial Intelligence) general embedding.

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
bge_embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='BAAI/bge-small-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ')

## **Preparing Data**

In [10]:
loaders = [
    TextLoader('/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('/content/blog_posts/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt'),
]
documents = []
for l in loaders:
    documents.extend(l.load())


In [11]:
len(documents)

2

In [12]:
documents[1]

Document(page_content='URL: https://blog.langchain.dev/benchmarking-question-answering-over-csv-data/\nTitle: Benchmarking Question/Answering Over CSV Data\n\nThis is a bit of a longer post. It\'s a deep dive on question-answering over tabular data. We discuss (and use) CSV data in this post, but a lot of the same ideas apply to SQL data. It covers:\n\nBackground Motivation: why this is an interesting task\n\nInitial Application: how we set up a simple Streamlit app in order to gather a good distribution of real questions\n\nInitial Solution: our initial solution and some conceptual considerations\n\nDebugging with LangSmith: what we saw people asking, and what issues our initial solution had\n\nEvaluation Setup: how we evaluated solutions\n\nImproved Solution: the final improved solution we arrived at\n\nAs a sneak preview, the improved solution we arrived at was a custom agent that used OpenAI functions and had access to two tools: a Python REPL and a retriever.\n\nWe\'ve open-source

## **Retrieving full documents rather than chunks**

 The initial full documents are not too big and we are not going to return many of them.

In [13]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [14]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="full_documents",
                      embedding_function=bge_embeddings  #OpenAIEmbeddings()
)


In [15]:
# The storage layer for the parent documents
store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(vectorstore=vectorstore,
                                             docstore=store,
                                             child_splitter=child_splitter,)

In [16]:
full_doc_retriever.add_documents(documents, ids=None)


##List the number of documents added

In [17]:
list(store.yield_keys())

['8d84507e-c4b8-495c-98fe-658a02b33cb7',
 '7d08e286-ed2f-46b5-adcf-0a0e77dccb94']

In [18]:
print(f"Number of documents added : {len(list(store.yield_keys()))}")

Number of documents added : 2


###Vector store search functionality

In [19]:
sub_documents = vectorstore.similarity_search("what is langsmith?")

In [20]:
sub_documents

[Document(page_content='Today, we’re introducing LangSmith, a platform to help developers close the gap between prototype and production. It’s designed for building and iterating on products that can harness the power–and wrangle the complexity–of LLMs.', metadata={'doc_id': '8d84507e-c4b8-495c-98fe-658a02b33cb7', 'source': '/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'}),
 Document(page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.', metadata={'doc_id': '8d84507e-c4b8-495c-98fe-658a02b33cb7', 'source': '/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'}),
 Document(page_content='LangSmith is now in closed beta. So if you’re looking for a robust, unified, system for debugging, testing, evaluating, and monitoring your LLM applic

In [21]:
len(sub_documents)

4

In [22]:
print(sub_documents[0].page_content)


Today, we’re introducing LangSmith, a platform to help developers close the gap between prototype and production. It’s designed for building and iterating on products that can harness the power–and wrangle the complexity–of LLMs.


In [23]:
print(sub_documents[1].page_content)

URL: https://blog.langchain.dev/announcing-langsmith/
Title: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications

LangChain exists to make it as easy as possible to develop LLM-powered applications.


In [24]:
print(sub_documents[2].page_content)

LangSmith is now in closed beta. So if you’re looking for a robust, unified, system for debugging, testing, evaluating, and monitoring your LLM applications, sign up here.



How did we get here?


In [25]:
print(sub_documents[3].page_content)

Fintual, a Latin American startup with big dreams to help their citizens build wealth through a personalized financial advisor, found LangSmith early in their LLM development journey. “As soon as we heard about LangSmith, we moved our entire development stack onto it. We could build the evaluation, testing, and monitoring tools we needed in-house. But it would be 1000x worse, take us 10x longer,



##Retrieve relevant documents


In [26]:
retrieved_docs = full_doc_retriever.get_relevant_documents("what is langsmith?")

In [27]:
len(retrieved_docs[0].page_content)

11652

In [28]:
len(retrieved_docs)


1

In [29]:
retrieved_docs[0].page_content

'URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.\n\nThe blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main issue that we see today 

##Retrieving larger chunks

The full documents can be too big to want to retrieve them as is. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [30]:
# This text splitter is used to create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents - The small chunks
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="split_parents", embedding_function=bge_embeddings) #OpenAIEmbeddings()

# The storage layer for the parent documents
store = InMemoryStore()



##Retrive larger chunks


In [31]:
big_chunks_retriever = ParentDocumentRetriever(vectorstore=vectorstore,
                                                docstore=store,
                                                child_splitter=child_splitter,
                                                parent_splitter=parent_splitter,)

In [32]:
big_chunks_retriever.add_documents(documents)

In [33]:
print(f"number of documents added : {len(list(store.yield_keys()))}")

number of documents added : 18


In [34]:
retrieved_docs = big_chunks_retriever.get_relevant_documents("what is langsmith?")

In [35]:
len(retrieved_docs)

2

In [36]:
len(retrieved_docs[0].page_content)

1869

In [37]:
print(retrieved_docs[0].page_content)


URL: https://blog.langchain.dev/announcing-langsmith/
Title: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications

LangChain exists to make it as easy as possible to develop LLM-powered applications.

We started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.

The blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main issue that we see today is appli

In [38]:
print(retrieved_docs[1].page_content)


“Thanks to Langchain smith we were able to analyze our LLM calls, understand the performance of the different chain methods ( stuff vs reduce) for QA and improve upon it. It even helped us debug and understand errors we made. We are consistently using it to improve our prompt engineering and look forward to the new features,” said Stan Girard, Head of GenAI at Theodo and creator of Quivr.



A unified platform

While each of these product areas provide unique value, often at a specific point in time in the development process, we believe a great deal of the long term impact of LangSmith will come from having a single, fully-integrated hub to do this work from. We see teams with all kinds of Rube Goldberg-machine-like processes for managing their LLM applications, and we want to make that a thing of the past.

As a very simple example, we considered it to be table stakes for LangSmith to help users easily create datasets from existing logs and use them immediately for testing and evalua

Generate response based on the content supplied by the LLM using RetrievalQA Chain

In [39]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                 chain_type="stuff",
                                 retriever=big_chunks_retriever)

In [40]:
query = "What is Langsmith?"
qa.run(query)

' LangSmith is a unified platform designed to help developers close the gap between prototype and production for applications that use a specific type of technology called LLMs.'

##Question 2

In [41]:
retriever_documents = big_chunks_retriever.get_relevant_documents("what is RAGAS")
print(len(retriever_documents))

2


In [42]:
print(retriever_documents[0].page_content)

URL: https://blog.langchain.dev/benchmarking-question-answering-over-csv-data/
Title: Benchmarking Question/Answering Over CSV Data

This is a bit of a longer post. It's a deep dive on question-answering over tabular data. We discuss (and use) CSV data in this post, but a lot of the same ideas apply to SQL data. It covers:

Background Motivation: why this is an interesting task

Initial Application: how we set up a simple Streamlit app in order to gather a good distribution of real questions

Initial Solution: our initial solution and some conceptual considerations

Debugging with LangSmith: what we saw people asking, and what issues our initial solution had

Evaluation Setup: how we evaluated solutions

Improved Solution: the final improved solution we arrived at

As a sneak preview, the improved solution we arrived at was a custom agent that used OpenAI functions and had access to two tools: a Python REPL and a retriever.

We've open-sourced everything - the app we used to gather fee

In [43]:
print(retriever_documents[1].page_content)

💡 Evaluation of LLM applications is often hard because of a lack of data and a lack of metrics.

In traditional machine learning you usually start with a dataset of inputs and outputs, and you use this to train and then evaluate your model. However, because LLMs are fantastic zero shot learners, it is now possible to use a prompt to quickly build an application based on just an idea, and no data. While this is incredibly powerful in terms of enabling developers to build new applications quickly, it leads to difficulty in evaluating because you lack that data. This is why we've built LangSmith in a way where constructing datasets is as easy as possible.

Likewise, there's often not great metrics for evaluating LLM applications. The outputs are often natural language, and traditional NLP metrics like BLEU and ROUGE aren't great. But what is good at understanding natural language? LLMs! We're pretty bullish on LLM assisted evaluation and have invested in building a bunch of evaluators tha

In [45]:
query = "What is RAGAS?"
qa.run(query)

'\nRAGAS stands for "Reinforcement learning for Automated Grid Analysis and Scoring". It is a framework for applying reinforcement learning techniques to optimize grids in power systems. '